## FINETUNED WAV2VEC

###### Import libraries

In [ ]:
!pip install pydub

In [ ]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

#### Prepare the Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!unzip "/content/gdrive/MyDrive/mozilla/Mozilla Foundation - Mozilla Common Voice Hackathon-20231013T080731Z-001.zip"

Archive:  /content/gdrive/MyDrive/mozilla/Mozilla Foundation - Mozilla Common Voice Hackathon-20231013T080731Z-001.zip
  inflating: Mozilla Foundation - Mozilla Common Voice Hackathon/test0.tar.gz  
  inflating: Mozilla Foundation - Mozilla Common Voice Hackathon/eval0.tar.gz  


In [ ]:
import tarfile
import os

# Create the directories if they don't exist
if not os.path.exists('datasets/train'):
    os.makedirs('datasets/train')


train_audio_path = "/content/gdrive/MyDrive/mozilla/train0-002.tar.gz"

# Unzip the train_audio_path to datasets/train
with tarfile.open(train_audio_path, 'r:gz') as train_tar:
    train_tar.extractall('datasets/train')




Unzipped audio files to respective folders.


In [ ]:

import pandas as pd

train = pd.read_csv("/content/gdrive/MyDrive/mozilla/train.tsv",sep = '\t' )
train.dropna(subset = ['age', 'gender'], inplace = True)
train = train[~train['path'].isin(["common_voice_sw_30035155.mp3", "common_voice_sw_30377114.mp3","common_voice_sw_30318282.mp3"])]

train['gender_age'] = train['gender'] + "_" + train['age']
train['sentence'] = train['sentence'].str.lower()

name = "alamsher/wav2vec2-large-xlsr-53-common-voice-sw"






In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_valid = train_test_split(train, test_size = 0.2, stratify = train['gender_age'], shuffle =True, random_state = 42)
df_valid = df_valid[~df_valid['path'].isin(['common_voice_sw_31818110.mp3'])]


df_train.sample(40000, random_state= 0).to_csv("df_train.csv", index=False)
df_valid.sample(10000, random_state= 42).to_csv("df_valid.csv", index=False)

display(df_train.head(), df_train.shape, df_valid.head(), df_valid.shape)




,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment,gender_age
61046,6e09ff9cc1f149df26e8db256a211d367bced4288641e2...,common_voice_sw_30058956.mp3,nchi hizi zipo mbili katika bara la afrika,2,0,twenties,female,NaN,NaN,sw,NaN,female_twenties
19063,4fe6974e93db06e39211066366a14a80c728d9b68ac774...,common_voice_sw_30586117.mp3,halmashauri ya jiji wanatumia umwagiliaji wa m...,2,1,thirties,male,NaN,NaN,sw,NaN,male_thirties
41476,7ec265470aa1c06d5b156e0c67c5014cfaf580e7074798...,common_voice_sw_28672985.mp3,udhibiti wa uzazi wa dharura unaweza kuua mimb...,2,0,twenties,female,NaN,NaN,sw,NaN,female_twenties
33725,c33143eb3cc55c56cd663f4e7fac612f7fa9d3b22450ec...,common_voice_sw_29923497.mp3,zilizo bora kabisa kote afrika mashariki na kati,2,0,twenties,female,NaN,NaN,sw,NaN,female_twenties
69673,6598f66f88154f63cff3693c80c067abd86ddc899776cd...,common_voice_sw_30308344.mp3,zilikuwa zimetokana na utu au nafsi yake si ch...,2,0,twenties,male,NaN,NaN,sw,NaN,male_twenties


(42129, 12)

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment,gender_age
54298,8424b9a64f71f7addfe36e56debfb9eac09dac2927bd65...,common_voice_sw_30334160.mp3,kuhusu wahusika kuna wahusika,2,0,thirties,male,NaN,NaN,sw,NaN,male_thirties
35983,e1335e83dada24bdbbfc0da0217e601ca37d2a0619fbcb...,common_voice_sw_30140079.mp3,kitaifa maadhimisho haya yalifanyika katika ki...,2,0,twenties,male,NaN,NaN,sw,NaN,male_twenties
34694,5dcf9513ac6684fa981e7aade4446b14676ab0822e6912...,common_voice_sw_30567404.mp3,idi amin alilisha maiti za maadui zake kwa mamba,3,1,sixties,female,NaN,NaN,sw,NaN,female_sixties
32758,6370d4536d7033b81e4957d1c9671efa1c82e1ac2dac6c...,common_voice_sw_35160902.mp3,unapokuwa katika kampuni jaribu kufuata ishara...,15,0,twenties,female,NaN,NaN,sw,NaN,female_twenties
11416,12e695579df22b96d7a8248070cb02054dfc2ac2aef88d...,common_voice_sw_34743504.mp3,na onyesho la hadharani la nane la mwaka,2,1,thirties,female,NaN,NaN,sw,NaN,female_thirties


(10532, 12)

#### Install Dependencies and Import lIbraries

In [ ]:
!pip install -qq datasets==2.1.0
!pip install -qq transformers==4.18.0
!pip install -qq jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 33.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 11.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import random
from datasets import load_dataset, load_metric, Audio
import re
import json
from transformers import TrainingArguments, Trainer
from transformers import AutoProcessor, AutoModelForCTC

import torch
import librosa
import IPython.display as ipd

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

#### Login to your HUgging Face account
* Needed to push your model to hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Here, we create a processor using AutoProcessor.from_pretrained(name), where name corresponds to the Swahili-specific ASR model we've defined earlier.

In [ ]:
processor = AutoProcessor.from_pretrained(name)

#### Data Cleaning Functions


In [ ]:
def load_data(data_files):
  "Load data using hugginface dataset.load_dataset()"
  data = load_dataset("csv", data_files=data_files)
  return data

chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\']'

def remove_special_characters(batch, column_name = "sentence"):
    batch[column_name] = re.sub(chars_to_remove_regex, '', batch[column_name])
    return batch

def replace_extra_characters(batch,column_name="sentence"):
    batch[column_name] = re.sub('[ç]', 'c', batch[column_name])
    batch[column_name] = re.sub('[é]', 'e', batch[column_name])
    batch[column_name] = re.sub('[...]', '', batch[column_name])
    batch[column_name] = re.sub('[\t]', '', batch[column_name])
    return batch

def extract_all_chars(batch,column_name="sentence"):
  all_text = " ".join(batch[column_name])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}



def speech_file_to_array_fn(batch, column_name="sentence", audio_folder=None):
    if audio_folder is None:
        audio_folder = "/content/datasets/train/train/"

    speech_array, sampling_rate = librosa.load(audio_folder + batch["path"], mono=True, sr=16000)
    batch["speech"] = np.array(speech_array, dtype=np.float32)
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch[column_name]
    return batch


def prepare_dataset(batch,column_name = "sentence"):
    # batch = batch["batch"]

    # batched output is "un-batched"
    batch["input_values"] = processor(np.array(batch["speech"]), sampling_rate=16000).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

#### DataCollatorSpeechSeq2SeqWithPadding

This data collator is designed to prepare the data for a sequence-to-sequence ASR task, ensuring that inputs and labels are correctly padded and processed for training.

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

#### Define the Metric

In [ ]:
wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metric
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import AutoProcessor, AutoModelForCTC

#### Data Preprocessing

In [ ]:
# ### STEP 1
data_files = {
   "train": "/content/df_train.csv",
   "valid": "/content/df_valid.csv",



             }
# # load data
data = load_data(data_files)

# # clean data
data["train"] = data["train"].map(remove_special_characters)
data["valid"] = data["valid"].map(remove_special_characters)


data["train"] = data["train"].map(replace_extra_characters)
data["valid"] = data["valid"].map(replace_extra_characters)


!mkdir "artifacts/"

data["train"] = data["train"].map(speech_file_to_array_fn,
                                remove_columns=data["train"].column_names)
data["valid"] = data["valid"].map(speech_file_to_array_fn,
                                remove_columns=data["valid"].column_names)

print("check data")
rand_int = random.randint(0, len(data)-1)
ipd.Audio(data=data["train"][rand_int]["speech"], autoplay=True, rate=16000)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f5a096132501660e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/40000 [00:00<?, ?ex/s]

  0%|          | 0/10000 [00:00<?, ?ex/s]

...

  0%|          | 0/10000 [00:00<?, ?ex/s]

mkdir: cannot create directory ‘artifacts/’: File exists


  0%|          | 0/40000 [00:00<?, ?ex/s]

  0%|          | 0/10000 [00:00<?, ?ex/s]

check data


In [ ]:
data["train"] = data["train"].map(prepare_dataset, remove_columns=data["train"].column_names)
data["valid"] = data["valid"].map(prepare_dataset, remove_columns=data["valid"].column_names)


  0%|          | 0/40000 [00:00<?, ?ex/s]

  0%|          | 0/10000 [00:00<?, ?ex/s]

#### Instantiate the Model

In [ ]:
model = AutoModelForCTC.from_pretrained(name)

Downloading:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

In [ ]:
model.freeze_feature_extractor()

#### Training Args

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
training_args = TrainingArguments(
      output_dir="./40k_10k_0.073_xlsr_sw",
        group_by_length=True,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=1,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=2e-05,
  warmup_steps=500,
  save_total_limit=2,
  load_best_model_at_end=True, # load model with highest F1 score
        # push_to_hub = True
  )

#### Training Whohoo

In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data["train"],
    eval_dataset=data["valid"],
    tokenizer=processor.feature_extractor,
  )
trainer.train()

Using amp half precision backend
The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 40000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 2500


Step,Training Loss,Validation Loss,Wer
500,0.213800,inf,0.077337
1000,0.122000,inf,0.074323
1500,0.125000,inf,0.073977
2000,0.121300,inf,0.073416
2500,0.116300,inf,0.073135


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
Saving model checkpoint to ./13k_4k/checkpoint-500
Configuration saved in ./13k_4k/checkpoint-500/config.json
Model weights saved in ./13k_4k/checkpoint-500/pytorch_model.bin
Feature extractor saved in ./13k_4k/checkpoint-500/preprocessor_config.json
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
Saving model checkpoint to ./13k_4k/checkpoint-1000
Configuration saved in ./13k_4k/checkpoint-1000/config.json
Model

TrainOutput(global_step=2500, training_loss=0.13969105682373048, metrics={'train_runtime': 10922.4764, 'train_samples_per_second': 3.662, 'train_steps_per_second': 0.229, 'total_flos': 6.870426129800893e+18, 'train_loss': 0.13969105682373048, 'epoch': 1.0})

#### Push your Model to HUb

In [ ]:
import os
import shutil

folder_path = "/content/40k_10k_0.073_xlsr_sw"  # Replace with the path to your folder

# Remove existing files (if you don't need them)
shutil.rmtree(folder_path, ignore_errors=True)




In [ ]:
trainer.push_to_hub()


Cloning https://huggingface.co/Koleshjr/13k_4k into local empty directory.
Saving model checkpoint to ./13k_4k
Configuration saved in ./13k_4k/config.json
Model weights saved in ./13k_4k/pytorch_model.bin
Feature extractor saved in ./13k_4k/preprocessor_config.json


Upload file pytorch_model.bin:   0%|          | 1.00/1.18G [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.00k [00:00<?, ?B/s]

To https://huggingface.co/Koleshjr/13k_4k
   fdfdf9d..5560fab  main -> main

   fdfdf9d..5560fab  main -> main

To https://huggingface.co/Koleshjr/13k_4k
   5560fab..7bb6388  main -> main

   5560fab..7bb6388  main -> main



'https://huggingface.co/Koleshjr/13k_4k/commit/5560fabf2285c78f2caadcafd45bb6bc0f495d5d'

In [ ]:
processor.push_to_hub("40k_10k_0.073_xlsr_sw")
processor.tokenizer.push_to_hub('40k_10k_0.073_xlsr_sw')
processor.feature_extractor.push_to_hub('40k_10k_0.073_xlsr_sw')

Feature extractor saved in 13k_4k/preprocessor_config.json
tokenizer config file saved in 13k_4k/tokenizer_config.json
Special tokens file saved in 13k_4k/special_tokens_map.json
added tokens file saved in 13k_4k/added_tokens.json
To https://huggingface.co/Koleshjr/13k_4k
   7bb6388..0f7f8b0  main -> main

   7bb6388..0f7f8b0  main -> main

tokenizer config file saved in 13k_4k/tokenizer_config.json
Special tokens file saved in 13k_4k/special_tokens_map.json
added tokens file saved in 13k_4k/added_tokens.json
Feature extractor saved in 13k_4k/preprocessor_config.json
